In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import re, os
import openai
import numpy as np
from typing import List

# 1. get all FIT5145 course participants

In [2]:

user_info_path = "../all_raw_data/2024S2/2024S2_user_id_email_student_number_map.txt" # this file is removed for anonymous purposes
df_user_info = pd.read_csv(user_info_path)
all_user_id_list = df_user_info['user_id'].tolist()
print(len(all_user_id_list))
print(all_user_id_list) # total 519 students

519
[3260, 3261, 3262, 3263, 3264, 3265, 3266, 3267, 3268, 3269, 3270, 3271, 3272, 3273, 3274, 3275, 3276, 3277, 3278, 3279, 3280, 3281, 3282, 3283, 3284, 3285, 3286, 3287, 3288, 3289, 3290, 3291, 3292, 3293, 3294, 3295, 3296, 3297, 3298, 3299, 3300, 3301, 3302, 3303, 3304, 3305, 3306, 3307, 3308, 3309, 3310, 3311, 3312, 3313, 3314, 3315, 3316, 3317, 3318, 3319, 3320, 3321, 3322, 3323, 3324, 3325, 3326, 3327, 3328, 3329, 3330, 3331, 3332, 3333, 3334, 3335, 3336, 3337, 3338, 3339, 3340, 3341, 3342, 3343, 3344, 3346, 3347, 3348, 3349, 3350, 3351, 3352, 3353, 3354, 3355, 3356, 3357, 3358, 3359, 3360, 3361, 3362, 3363, 3364, 3365, 3366, 3367, 3368, 3369, 3370, 3371, 3372, 3373, 3374, 3375, 3376, 3377, 3378, 3379, 3380, 3381, 3382, 3383, 3384, 3385, 3386, 3387, 3388, 3389, 3390, 3391, 3392, 3393, 3394, 3395, 3396, 3397, 3398, 3399, 3400, 3401, 3402, 3403, 3404, 3405, 3406, 3407, 3408, 3409, 3410, 3411, 3412, 3413, 3414, 3415, 3416, 3417, 3418, 3419, 3420, 3421, 3422, 3423, 3424, 3425, 3426,

# 2. Get all users who have submitted the assignment and has writing log in FLoRA

In [3]:
student_proposal_txt_folder_path = "../all_raw_data/2024S2/student_proposal_txt" # students original proposal submissions
user_info_path = "../all_raw_data/2024S2/2024S2_user_id_email_student_number_map.txt" # this file is removed for anonymous purposes
final_proposal_on_flora_path = '../all_raw_data/2024S2/flora_annotation_final_essay.xlsx' # student's proposal recorded in the FLoRA system (This is open sourced)
df_user_info = pd.read_csv(user_info_path)
df_final_proposal_on_flora = pd.read_excel(final_proposal_on_flora_path)
all_user_id_list = []
for file in os.listdir(student_proposal_txt_folder_path):
    if not file.endswith('.txt'):
        # print("invalid filename:", file)
        continue
    # Get ID number, email and user_id for each row
    id_number = Path(file).stem
    col_int = pd.to_numeric(df_user_info['ID number'], errors='coerce')
    result = df_user_info.loc[col_int == int(id_number)]

    if result.empty:
        # print(f"id number not exist in participants file: id_number: {id_number}, file: {file}")
        continue

    email = result['Email address'].iloc[0]
    user_id = int(result['user_id'].iloc[0])

    # Find the corresponding essay_content in df_final_proposal_on_flora based on user_id
    proposal_result = df_final_proposal_on_flora.loc[df_final_proposal_on_flora['user_id'] == user_id, 'essay_content']
    if proposal_result.empty:
        # print(f"student submit proposal but not use FLoRA environment: id_number: {id_number}, user_id: {user_id}, email: {email}, file: {file}")
        continue
    all_user_id_list.append(user_id)

print(f"All --- Number of users: {len(all_user_id_list)}")
print(all_user_id_list) # 337 students left

All --- Number of users: 337
[3562, 3579, 3699, 3626, 3650, 3325, 3457, 3702, 3526, 3599, 3432, 3772, 3678, 3284, 3374, 3525, 3474, 3700, 3578, 3729, 3451, 3742, 3661, 3459, 3685, 3753, 3715, 3723, 3524, 3566, 3659, 3312, 3670, 3445, 3411, 3646, 3682, 3398, 3719, 3472, 3532, 3396, 3386, 3393, 3701, 3316, 3331, 3669, 3467, 3542, 3586, 3489, 3603, 3649, 3714, 3775, 3638, 3757, 3493, 3694, 3395, 3677, 3360, 3273, 3274, 3674, 3530, 3308, 3399, 3487, 3301, 3707, 3776, 3684, 3436, 3676, 3596, 3507, 3389, 3486, 3576, 3616, 3280, 3732, 3271, 3492, 3293, 3447, 3752, 3609, 3703, 3499, 3754, 3761, 3464, 3413, 3533, 3513, 3628, 3460, 3536, 3522, 3455, 3763, 3760, 3311, 3351, 3431, 3636, 3681, 3605, 3384, 3564, 3452, 3523, 3709, 3365, 3739, 3634, 3698, 3541, 3683, 3397, 3607, 3289, 3620, 3668, 3470, 3751, 3295, 3385, 3557, 3712, 3755, 3689, 3680, 3642, 3664, 3597, 3480, 3388, 3539, 3329, 3601, 3406, 3687, 3412, 3490, 3691, 3326, 3458, 3726, 3728, 3735, 3627, 3555, 3572, 3565, 3771, 3404, 3654, 3449

# 3. Filter Non-English students

In [14]:
from transformers import pipeline, AutoTokenizer
import nltk

# Download punkt if using for the first time
nltk.download('punkt')

chat_log_path = "../all_raw_data/2024S2/flora_annotation_user_chatgpt_log_2024S2.xlsx"

langid_pipe = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")


from concurrent.futures import ThreadPoolExecutor

def process_row(row):
    id = row['id']
    user_id = row['user_id']
    user_questions = row['user_questions']
    has_non_eng_text, result_set = has_non_english(user_questions)
    if has_non_eng_text:
        result = f'id:{id}---------user_id: {user_id}------------------{user_questions}---------------{result_set}\n\n'
        return (user_id, result)
    else:
        return None


def has_non_english(text):
    sentences = nltk.sent_tokenize(text)
    results = []
    for sent in sentences:
        if len(sent) > 512:
            sent = sent[0:510]
        results.append(langid_pipe(sent)[0]["label"])

    results_set = set(results)
    if "zh" not in results_set:
        return False, results_set
    else:
        return True, results_set
    # if len(results_set) == 1 and "en" in results_set:
    #     return False, results_set
    # else:
    #     return True, results_set

"""
def detect_language_each_user():


    chat_df = pd.read_excel(chat_log_path)
    non_eng_result = ""
    non_eng_user_id_list = []
    for idx, row in chat_df.iterrows():
        print(idx)
        id = row['id']
        user_id = row['user_id']
        user_questions = row['user_questions']
        has_non_eng_text, result_set = has_non_english(user_questions)
        if has_non_eng_text:
            non_eng_result += f'id:{id}---------user_id: {user_id}------------------{user_questions}---------------{result_set}\n\n'
            non_eng_user_id_list.append(user_id)
        else:
            continue

    with open("./non_eng_result2.txt", "w", encoding="utf8") as f:
        f.write(non_eng_result)

    return non_eng_user_id_list
"""
chat_df = pd.read_excel(chat_log_path)
rows = chat_df.to_dict('records')  # Recommended approach: get list of row dicts
non_eng_result = ""
non_eng_user_id_list = []

with ThreadPoolExecutor(max_workers=20) as executor:
    results = executor.map(process_row, rows)

for item in results:
    if item:
        user_id, result = item
        non_eng_result += result
        non_eng_user_id_list.append(user_id)
user_has_non_english_list = non_eng_user_id_list
# user_has_non_english_list = detect_language_each_user()
print(len(user_has_non_english_list))
print(user_has_non_english_list) # [3283, 3291, 3300, 3307, 3312, 3389, 3391, 3481, 3502, 3514, 3516, 3525, 3529, 3533, 3569, 3615, 3636, 3640, 3667, 3671, 3693, 3694, 3736, 3739, 3741, 3742, 3759]
# user_has_non_english_list = []
filter_result_after_step3 = [x for x in all_user_id_list if x not in set(user_has_non_english_list)]


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Xinyu\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Device set to use cpu


49
[3283, 3283, 3291, 3300, 3307, 3307, 3307, 3307, 3312, 3312, 3312, 3389, 3391, 3481, 3502, 3502, 3514, 3516, 3516, 3516, 3516, 3516, 3516, 3525, 3529, 3533, 3533, 3533, 3533, 3533, 3569, 3615, 3636, 3667, 3671, 3693, 3694, 3694, 3736, 3739, 3741, 3742, 3742, 3742, 3742, 3742, 3742, 3742, 3759]


In [33]:

user_has_non_english_list = [3283, 3291, 3300, 3307, 3312, 3389, 3391, 3481, 3502, 3514,
     3516, 3525, 3529, 3533, 3569, 3615, 3636, 3640, 3667, 3671,
     3693, 3694, 3736, 3739, 3741, 3742, 3759]
filter_result_after_step3 = [x for x in all_user_id_list if x not in set(user_has_non_english_list)]
print(len(filter_result_after_step3))
print(filter_result_after_step3)

320
[3562, 3579, 3699, 3626, 3650, 3325, 3457, 3702, 3526, 3599, 3432, 3772, 3678, 3284, 3374, 3474, 3700, 3578, 3729, 3451, 3661, 3459, 3685, 3753, 3715, 3723, 3524, 3566, 3659, 3670, 3445, 3411, 3646, 3682, 3398, 3719, 3472, 3532, 3396, 3386, 3393, 3701, 3316, 3331, 3669, 3467, 3542, 3586, 3489, 3603, 3649, 3714, 3775, 3638, 3757, 3493, 3395, 3677, 3360, 3273, 3274, 3674, 3530, 3308, 3399, 3487, 3301, 3707, 3776, 3684, 3436, 3676, 3596, 3507, 3486, 3576, 3616, 3280, 3732, 3271, 3492, 3293, 3447, 3752, 3609, 3703, 3499, 3754, 3761, 3464, 3413, 3513, 3628, 3460, 3536, 3522, 3455, 3763, 3760, 3311, 3351, 3431, 3681, 3605, 3384, 3564, 3452, 3523, 3709, 3365, 3634, 3698, 3541, 3683, 3397, 3607, 3289, 3620, 3668, 3470, 3751, 3295, 3385, 3557, 3712, 3755, 3689, 3680, 3642, 3664, 3597, 3480, 3388, 3539, 3329, 3601, 3406, 3687, 3412, 3490, 3691, 3326, 3458, 3726, 3728, 3735, 3627, 3555, 3572, 3565, 3771, 3404, 3654, 3449, 3613, 3440, 3387, 3758, 3355, 3748, 3471, 3496, 3655, 3662, 3518, 3711,

In [4]:
from langdetect import detect, DetectorFactory
import pandas as pd
import re, os
from transformers import pipeline
chat_log_path = "../all_raw_data/2024S2/flora_annotation_user_chatgpt_log_2024S2.xlsx"
# Load pretrained "text classification" offensive content detection model
classifier = pipeline("text-classification", model="unitary/toxic-bert")

def is_offensive_with_transformers(text):
    results = classifier(text[0:500])
    for result in results:
        # Check if classified as "toxic"
        if result['label'].lower() == 'toxic' and result['score'] > 0.5:  # custom threshold
            return True
    return False

def contains_eight_digit_number(text):
    return bool(re.search(r'\b\d{8}\b', text))

def contains_email(text):
    email_regex = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
    return bool(re.search(email_regex, text))

# session_chat_path = "../all_raw_data/output_with_conversation_sessions_27min.xlsx"
# df_session_chat = pd.read_excel(session_chat_path)
# df_session_chat['user_ask_time'] = pd.to_datetime(df_session_chat['user_ask_time'], unit='ms')
# df_session_filtered = df_session_chat.loc[df_session_chat['user_id'].isin(filter_non_engage_user_id_list)]
chat_df = pd.read_excel(chat_log_path)
sss = ""
for index, row in chat_df.iterrows():
    question = row['user_questions']
    answer = row['chatgpt_answer']

    # Check for offensive content
    if is_offensive_with_transformers(question):
        sss += f"offensive-----id-----{row['id']}-----user-id-----{row['user_id']}-----questions-----{row['user_questions']}\n"
    # Check if it contains 8-digit numbers, email, or student name
    if contains_email(question):
        sss += f"contains email-----id-----{row['id']}-----user-id-----{row['user_id']}-----questions-----{row['user_questions']}\n"
    if contains_eight_digit_number(question):
        sss += f"contains eightdigit-----id-----{row['id']}-----user-id-----{row['user_id']}-----questions-----{row['user_questions']}\n"
with open("offensive_info.txt", "w", encoding="utf8") as f:
    f.write(sss)

Device set to use cpu


# 4. Filter users who have <= 1 utterance

In [34]:
chat_log_path = "../all_raw_data/2024S2/flora_annotation_user_chatgpt_log_2024S2.xlsx"
chat_df = pd.read_excel(chat_log_path)
user_counts = chat_df.groupby('user_id').size()
user_no_chat_log = [x for x in filter_result_after_step3 if x not in set(user_counts.index.tolist())]
print(len(user_no_chat_log)) #
print(user_no_chat_log) # [3698, 3630, 3438, 3558, 3509, 3679]
user_counts_less_than_one = user_counts[user_counts <= 1]
print(user_counts_less_than_one.index.tolist()) # [3333, 3354, 3385, 3414, 3441, 3454, 3527, 3541, 3614, 3625, 3667, 3709, 3727, 3732]
filter_result_after_step4 = [x for x in filter_result_after_step3 if x not in set(user_counts_less_than_one.index.tolist() + user_no_chat_log)]
print(len(filter_result_after_step4))
print(filter_result_after_step4)

6
[3698, 3630, 3438, 3558, 3509, 3679]
[3333, 3354, 3385, 3414, 3441, 3454, 3527, 3541, 3614, 3625, 3667, 3709, 3727, 3732]
309
[3562, 3579, 3699, 3626, 3650, 3325, 3457, 3702, 3526, 3599, 3432, 3772, 3678, 3284, 3374, 3474, 3700, 3578, 3729, 3451, 3661, 3459, 3685, 3753, 3715, 3723, 3524, 3566, 3659, 3670, 3445, 3411, 3646, 3682, 3398, 3719, 3472, 3532, 3396, 3386, 3393, 3701, 3316, 3331, 3669, 3467, 3542, 3586, 3489, 3603, 3649, 3714, 3775, 3638, 3757, 3493, 3395, 3677, 3360, 3273, 3274, 3674, 3530, 3308, 3399, 3487, 3301, 3707, 3776, 3684, 3436, 3676, 3596, 3507, 3486, 3576, 3616, 3280, 3271, 3492, 3293, 3447, 3752, 3609, 3703, 3499, 3754, 3761, 3464, 3413, 3513, 3628, 3460, 3536, 3522, 3455, 3763, 3760, 3311, 3351, 3431, 3681, 3605, 3384, 3564, 3452, 3523, 3365, 3634, 3683, 3397, 3607, 3289, 3620, 3668, 3470, 3751, 3295, 3557, 3712, 3755, 3689, 3680, 3642, 3664, 3597, 3480, 3388, 3539, 3329, 3601, 3406, 3687, 3412, 3490, 3691, 3326, 3458, 3726, 3728, 3735, 3627, 3555, 3572, 3565, 3

# 5. Filter users whose similarity of Monash Moodle proposal and FLoRA recorded proposal <= 0.85

In [35]:

# Set your OpenAI API key
openai.api_key = "xxxxxxxxxxxxxxxx"  # Replace with your OpenAI API key

# Function to get embedding from OpenAI's text-embedding-3-small model
def get_embedding(text: str, model: str = "text-embedding-3-small") -> List[float]:
    # Optional: remove leading/trailing spaces and limit length
    text = text.strip().replace("\n", " ")

    response = openai.embeddings.create(
        model=model,
        input=[text]  # wrap in list for batch input
    )
    return response.data[0].embedding

# Function to compute cosine similarity between two vectors
def cosine_similarity(vec1, vec2):
    vec1 = np.array(vec1)
    vec2 = np.array(vec2)
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

def read_txt_content(file_path):
    with open(file_path, mode="r", encoding="utf8") as file:
        content = file.read()
        return content


final_proposal_on_flora_path = '../all_raw_data/2024S2/flora_annotation_final_essay.xlsx'
student_proposal_txt_folder_path = "../all_raw_data/2024S2/student_proposal_txt"
user_info_path = "../all_raw_data/2024S2/2024S2_user_id_email_student_number_map.txt"

df_user_info = pd.read_csv(user_info_path)
df_final_proposal_on_flora = pd.read_excel(final_proposal_on_flora_path)

cosine_sim_result = dict()

for file in os.listdir(student_proposal_txt_folder_path):
    if not file.endswith('.txt'):
        # print("invalid filename:", file)
        continue
    # Get ID number, email and user_id for each row
    id_number = Path(file).stem
    col_int = pd.to_numeric(df_user_info['ID number'], errors='coerce')
    result = df_user_info.loc[col_int == int(id_number)]

    if result.empty:
        # print(f"id number not exist in participants file: id_number: {id_number}, file: {file}")
        continue

    email = result['Email address'].iloc[0]
    user_id = int(result['user_id'].iloc[0])

    if user_id not in filter_result_after_step4:
        # print(f"user_id {user_id} not in filter_result2")
        continue

    txt_file_path = student_proposal_txt_folder_path + '/' + file
    txt_file_content = read_txt_content(txt_file_path)

    # Find the corresponding essay_content in df_final_proposal_on_flora based on user_id
    proposal_result = df_final_proposal_on_flora.loc[df_final_proposal_on_flora['user_id'] == user_id, 'essay_content']
    if proposal_result.empty:
        # print(f"student submit proposal but not use FLoRA environment: id_number: {id_number}, user_id: {user_id}, email: {email}, file: {file}")
        continue

    proposal_content = proposal_result.values[0]

    # use OpenAI embedding
    # Get embeddings
    embedding1 = get_embedding(txt_file_content)
    embedding2 = get_embedding(proposal_content)

    # Compute similarity
    similarity_score = cosine_similarity(embedding1, embedding2)
    cosine_sim_result[str(id_number) + "---" + str(user_id)] = similarity_score

print("--------result-------------------------------------------------------------------")

cosine_sim_sorted_result = dict(sorted(cosine_sim_result.items(), key=lambda item: item[1]))

print(cosine_sim_sorted_result)

filtered_result = {key: value for key, value in cosine_sim_sorted_result.items() if value > 0.85}
filtered_user_id_list = [int(key.split("---")[1]) for key, value in cosine_sim_sorted_result.items() if value > 0.85]

# print(filtered_result)

print(filtered_user_id_list)
print(len(filtered_user_id_list))

--------result-------------------------------------------------------------------
{'28408071---3626': 0.15420264969271, '34791086---3418': 0.45593781788518384, '33763356---3642': 0.4564190179413414, '34340955---3734': 0.5026973103164629, '34552278---3348': 0.5190290217618234, '35416289---3621': 0.5795472910427915, '32223846---3399': 0.6305283743294833, '33311625---3513': 0.7079993857640223, '33724040---3712': 0.7256328516698491, '34712518---3504': 0.7299876084607646, '34051201---3404': 0.7527734679003335, '35340436---3706': 0.7624176538547686, '35224193---3422': 0.7708073798719677, '31108121---3316': 0.7813743533114672, '29627958---3374': 0.7912258172392166, '33569479---3634': 0.8109224588996314, '34675523---3377': 0.8151345603871124, '31981259---3360': 0.8193167915025817, '33620229---3607': 0.8248591256620309, '32204078---3308': 0.826729917847135, '34332847---3580': 0.8390993009926817, '34425365---3344': 0.847021993483107, '34024891---3572': 0.854700875398067, '33529183---3384': 0.857

In [2]:
final_filtered_user_id_list = [3572, 3384, 3579, 3582, 3446, 3672, 3711, 3470, 3612, 3386, 3397, 3761, 3329, 3546, 3548, 3566, 3654, 3536, 3520, 3624, 3353, 3563, 3542, 3592, 3387, 3301, 3365, 3760, 3357, 3476, 3477, 3775, 3656, 3420, 3678, 3380, 3719, 3764, 3655, 3505, 3392, 3369, 3644, 3662, 3641, 3648, 3703, 3611, 3661, 3361, 3573, 3652, 3423, 3687, 3358, 3444, 3456, 3771, 3700, 3763, 3429, 3340, 3271, 3355, 3599, 3721, 3553, 3752, 3485, 3436, 3407, 3570, 3331, 3571, 3772, 3617, 3776, 3692, 3403, 3605, 3487, 3342, 3406, 3664, 3632, 3293, 3499, 3451, 3755, 3723, 3699, 3551, 3597, 3753, 3320, 3708, 3554, 3549, 3440, 3458, 3649, 3519, 3347, 3562, 3757, 3705, 3643, 3390, 3726, 3701, 3620, 3351, 3415, 3467, 3744, 3576, 3367, 3464, 3373, 3544, 3289, 3663, 3547, 3402, 3622, 3682, 3540, 3585, 3658, 3645, 3691, 3659, 3539, 3530, 3545, 3651, 3496, 3428, 3689, 3715, 3668, 3616, 3587, 3370, 3681, 3669, 3388, 3674, 3578, 3350, 3498, 3450, 3685, 3603, 3416, 3695, 3413, 3459, 3480, 3779, 3393, 3593, 3338, 3326, 3748, 3409, 3555, 3565, 3714, 3280, 3602, 3445, 3372, 3475, 3606, 3424, 3379, 3517, 3561, 3472, 3707, 3394, 3460, 3557, 3356, 3677, 3507, 3457, 3758, 3628, 3613, 3343, 3756, 3474, 3295, 3521, 3396, 3411, 3683, 3482, 3600, 3552, 3725, 3635, 3680, 3412, 3492, 3559, 3650, 3471, 3735, 3609, 3647, 3501, 3442, 3627, 3601, 3493, 3497, 3633, 3670, 3686, 3729, 3610, 3448, 3575, 3511, 3488, 3728, 3449, 3646, 3638, 3452, 3426, 3483, 3684, 3432, 3368, 3608, 3398, 3676, 3596, 3673, 3274, 3590, 3273, 3455, 3738, 3598, 3284, 3524, 3518, 3556, 3537, 3362, 3586, 3490, 3568, 3500, 3534, 3489, 3591, 3447, 3702, 3532, 3523, 3751, 3713, 3588, 3325, 3697, 3466, 3378, 3382, 3604, 3352, 3395, 3564, 3660, 3666, 3311, 3431, 3522, 3400, 3526, 3754, 3486] # 287

# Remove 8 students from the training session
# 279 remaining
final_filtered_user_id_list = [3572, 3384, 3579, 3582, 3446, 3672, 3711, 3470, 3612, 3386, 3397, 3761, 3329, 3546, 3548, 3566, 3654, 3536, 3520, 3624, 3353, 3563, 3542, 3592, 3387, 3301, 3365, 3760, 3357, 3476, 3477, 3775, 3656, 3420, 3678, 3380, 3719, 3764, 3655, 3505, 3392, 3644, 3662, 3641, 3648, 3703, 3611, 3661, 3361, 3573, 3652, 3423, 3687, 3358, 3444, 3456, 3771, 3700, 3763, 3429, 3340, 3271, 3355, 3599, 3721, 3553, 3752, 3485, 3436, 3407, 3570, 3331, 3571, 3772, 3617, 3776, 3692, 3403, 3605, 3487, 3342, 3406, 3664, 3632, 3293, 3499, 3451, 3755, 3723, 3699, 3551, 3597, 3753, 3320, 3708, 3554, 3549, 3440, 3458, 3649, 3519, 3347, 3562, 3757, 3705, 3643, 3390, 3726, 3701, 3620, 3351, 3415, 3467, 3744, 3576, 3367, 3464, 3373, 3544, 3289, 3547, 3402, 3622, 3682, 3540, 3585, 3658, 3645, 3691, 3659, 3539, 3530, 3545, 3651, 3496, 3428, 3689, 3715, 3668, 3616, 3587, 3370, 3681, 3669, 3388, 3674, 3578, 3350, 3498, 3450, 3685, 3603, 3416, 3695, 3413, 3459, 3480, 3779, 3393, 3338, 3326, 3748, 3409, 3555, 3565, 3714, 3280, 3602, 3445, 3372, 3475, 3606, 3424, 3379, 3517, 3472, 3707, 3394, 3460, 3557, 3356, 3677, 3507, 3457, 3628, 3613, 3343, 3756, 3474, 3295, 3521, 3396, 3411, 3683, 3482, 3600, 3552, 3725, 3635, 3680, 3412, 3492, 3559, 3471, 3735, 3609, 3647, 3501, 3442, 3627, 3601, 3493, 3497, 3633, 3670, 3686, 3729, 3610, 3448, 3575, 3511, 3488, 3728, 3449, 3646, 3638, 3452, 3426, 3483, 3684, 3432, 3608, 3398, 3676, 3596, 3673, 3274, 3590, 3273, 3455, 3738, 3598, 3284, 3524, 3518, 3537, 3362, 3586, 3490, 3568, 3500, 3534, 3489, 3591, 3447, 3702, 3532, 3523, 3751, 3713, 3588, 3325, 3697, 3466, 3378, 3382, 3604, 3352, 3395, 3564, 3660, 3666, 3311, 3431, 3522, 3400, 3526, 3754, 3486]

# annotation    2121 records
# chat log      3567 records  course id = 26
# trace log     7,579,722 records
# writing log   496,733 records
# number of students with trace data: 454


# 6. use 95% CI to split conversation session

In [4]:
# find 95% CI value
import numpy as np
import pandas as pd

def calculate_time_intervals(group, column_name):
    group = group.sort_values(column_name)
    intervals = group[column_name].diff().dt.total_seconds()
    return intervals



def get_95_CI(chat_file, user_id_list):
    df_raw_chat = pd.read_excel(chat_file)
    df_raw_chat['user_ask_time'] = pd.to_datetime(df_raw_chat['user_ask_time'], unit='ms')

    df_raw_chat_filtered = df_raw_chat.loc[df_raw_chat['user_id'].isin(user_id_list)]

    time_intervals = df_raw_chat_filtered.groupby('user_id').apply(calculate_time_intervals, column_name='user_ask_time').reset_index(level=0, drop=True)
    df_raw_chat_filtered['time_interval'] = time_intervals.fillna(0).astype('float64')

    df_chat_interval = df_raw_chat_filtered.dropna(subset=['time_interval'])

    Q1 = df_chat_interval['time_interval'].quantile(0.25)
    Q3 = df_chat_interval['time_interval'].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_filtered = df_chat_interval[(df_chat_interval['time_interval'] >= lower_bound) & (df_chat_interval['time_interval'] <= upper_bound)]
    confidence_95_upper = np.percentile(df_filtered['time_interval'], 95)
    print(f"95% CI: {confidence_95_upper:.2f} seconds")


raw_chat_path = '../all_raw_data/2024S2/flora_annotation_user_chatgpt_log_2024S2.xlsx'
filtered_user_id_list = final_filtered_user_id_list # from step 5
get_95_CI(raw_chat_path, filtered_user_id_list)


95% CI：2117.82 seconds


C:\Users\Xinyu\AppData\Local\Temp\ipykernel_12780\4080839806.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  time_intervals = df_raw_chat_filtered.groupby('user_id').apply(calculate_time_intervals, column_name='user_ask_time').reset_index(level=0, drop=True)
C:\Users\Xinyu\AppData\Local\Temp\ipykernel_12780\4080839806.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_raw_chat_filtered['time_interval'] = time_intervals.fillna(0).astype('float64')


In [4]:
# assign session number to each utterance
import pandas as pd
import numpy as np

def assign_session(user_df, column_name_time, column_name_block, threshold):
    user_df = user_df.sort_values(column_name_time).reset_index(drop=True)
    user_df['time_diff'] = user_df[column_name_time].diff().dt.total_seconds() / 60.0

    user_df[column_name_block] = 0
    block_counter = 0

    for i in range(1, len(user_df)):
        if user_df.loc[i, 'time_diff'] > threshold:  #1940.39/60
            block_counter += 1
        user_df.loc[i, column_name_block] = block_counter

    user_df = user_df.drop(columns=['time_diff'])
    return user_df


def generate_conversation_sessions(excel_file, user_id_list, threshold, result_file):
    df = pd.read_excel(excel_file)

    df['user_ask_time'] = pd.to_datetime(df['user_ask_time'], unit='ms')
    df_filtered = df
    if user_id_list:
        df_filtered = df.loc[df['user_id'].isin(user_id_list)]

    df_filtered = df_filtered.sort_values(by=['user_id', 'user_ask_time'])
    df_filtered = df_filtered.groupby('user_id').apply(assign_session, column_name_time='user_ask_time', column_name_block='conversation_session', threshold=threshold).reset_index(drop=True)
    df_filtered.to_excel(result_file, index=False)

raw_chat_path = '../all_raw_data/2024S2/flora_annotation_user_chatgpt_log_2024S2.xlsx'
filtered_user_id_list = final_filtered_user_id_list # from step 5
threshold_95_CI = 2117.82 / 60
generate_conversation_sessions(raw_chat_path, filtered_user_id_list, threshold_95_CI, '../all_raw_data/output_with_conversation_sessions_35min.xlsx')

C:\Users\Xinyu\AppData\Local\Temp\ipykernel_28080\2687993473.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_filtered = df_filtered.groupby('user_id').apply(assign_session, column_name_time='user_ask_time', column_name_block='conversation_session', threshold=threshold).reset_index(drop=True)


# 7. trace data distribution analysis (prove the platform work well)

In [ ]:
import matplotlib.pyplot as plt

# analyse the distribution of trace data

data1 = pd.read_excel('extra/FIT5145_2024_semester_1_trace_log_count.xlsx')
data2 = pd.read_excel('extra/FIT5145_2024_semester_2_trace_log_count.xlsx')

stats1 = data1['count(*)'].describe()
stats2 = data2['count(*)'].describe()

print('data1:', stats1)
print('data2:', stats2)

# histogram
plt.figure(figsize=(6, 4))

plt.hist(data1['count(*)'], bins=30, alpha=0.5, label='2024S1')
plt.hist(data2['count(*)'], bins=30, alpha=0.5, label='2024S2')
plt.xlabel('number of trace logs')
plt.ylabel('number of students')
plt.legend()
plt.grid(False)
# plt.savefig(fname="saved_figures/S1_S2 data distribution.png", dpi=300)
plt.show()

# box plot
plt.figure(figsize=(6, 4))
plt.boxplot([data1['count(*)'], data2['count(*)']], labels=['2024S1', '2024S2'])
plt.ylabel('number of trace logs')
plt.grid(False)
# plt.savefig(fname="saved_figures/S1_S2 data distribution_box plot.png", dpi=300)
plt.show()

# 7. Generate Data for label-studio (delete when open-source)

In [9]:
import random

final_filtered_user_id_list = [3572, 3384, 3579, 3582, 3446, 3672, 3711, 3470, 3612, 3386, 3397, 3761, 3329, 3546, 3548, 3566, 3654, 3536, 3520, 3624, 3353, 3563, 3542, 3592, 3387, 3301, 3365, 3760, 3357, 3476, 3477, 3775, 3656, 3420, 3678, 3380, 3719, 3764, 3655, 3505, 3392, 3369, 3644, 3662, 3641, 3648, 3703, 3611, 3661, 3361, 3573, 3652, 3423, 3687, 3358, 3444, 3456, 3771, 3700, 3763, 3429, 3340, 3271, 3355, 3599, 3721, 3553, 3752, 3485, 3436, 3407, 3570, 3331, 3571, 3772, 3617, 3776, 3692, 3403, 3605, 3487, 3342, 3406, 3664, 3632, 3293, 3499, 3451, 3755, 3723, 3699, 3551, 3597, 3753, 3320, 3708, 3554, 3549, 3440, 3458, 3649, 3519, 3347, 3562, 3757, 3705, 3643, 3390, 3726, 3701, 3620, 3351, 3415, 3467, 3744, 3576, 3367, 3464, 3373, 3544, 3289, 3663, 3547, 3402, 3622, 3682, 3540, 3585, 3658, 3645, 3691, 3659, 3539, 3530, 3545, 3651, 3496, 3428, 3689, 3715, 3668, 3616, 3587, 3370, 3681, 3669, 3388, 3674, 3578, 3350, 3498, 3450, 3685, 3603, 3416, 3695, 3413, 3459, 3480, 3779, 3393, 3593, 3338, 3326, 3748, 3409, 3555, 3565, 3714, 3280, 3602, 3445, 3372, 3475, 3606, 3424, 3379, 3517, 3561, 3472, 3707, 3394, 3460, 3557, 3356, 3677, 3507, 3457, 3758, 3628, 3613, 3343, 3756, 3474, 3295, 3521, 3396, 3411, 3683, 3482, 3600, 3552, 3725, 3635, 3680, 3412, 3492, 3559, 3650, 3471, 3735, 3609, 3647, 3501, 3442, 3627, 3601, 3493, 3497, 3633, 3670, 3686, 3729, 3610, 3448, 3575, 3511, 3488, 3728, 3449, 3646, 3638, 3452, 3426, 3483, 3684, 3432, 3368, 3608, 3398, 3676, 3596, 3673, 3274, 3590, 3273, 3455, 3738, 3598, 3284, 3524, 3518, 3556, 3537, 3362, 3586, 3490, 3568, 3500, 3534, 3489, 3591, 3447, 3702, 3532, 3523, 3751, 3713, 3588, 3325, 3697, 3466, 3378, 3382, 3604, 3352, 3395, 3564, 3660, 3666, 3311, 3431, 3522, 3400, 3526, 3754, 3486]

# labelled_user_id_list = [3588, 3590, 3591, 3593, 3597, 3598, 3599, 3600, 3601, 3602, 3603, 3604, 3605, 3616, 3624, 3627, 3636, 3640, 3643, 3644, 3645, 3646, 3647, 3648, 3651, 3652, 3654, 3655, 3656, 3661, 3670, 3671, 3672, 3673, 3676, 3677, 3683, 3684, 3687, 3692, 3694, 3695, 3699, 3700, 3701, 3702, 3705, 3707, 3714, 3715, 3719, 3721, 3726, 3729, 3738, 3739, 3742, 3748, 3751, 3752, 3754, 3755, 3759, 3763, 3767, 3771, 3779, 3338, 3342, 3343, 3347, 3350, 3351, 3352, 3353, 3355, 3356, 3357, 3358, 3360, 3361, 3362, 3367, 3368, 3369, 3370, 3378, 3387, 3388, 3389, 3393, 3395, 3396, 3398, 3400, 3402, 3406, 3407, 3411, 3413, 3415, 3416, 3426, 3428, 3432, 3440, 3442, 3446, 3447, 3449, 3450, 3451, 3455, 3456, 3457, 3458, 3459, 3460, 3464, 3467, 3470, 3472, 3474, 3476, 3477, 3480, 3483, 3487, 3489, 3492, 3497, 3499, 3500, 3501, 3507, 3511, 3517, 3520, 3521, 3523, 3526, 3533, 3534, 3536, 3540, 3545, 3546, 3547, 3548, 3549, 3551, 3552, 3553, 3554, 3557, 3559, 3562, 3563, 3564, 3568, 3569, 3570, 3571, 3575, 3576, 3578, 3579, 3582]
#
#
# final_filtered_user_id_list = [x for x in final_filtered_user_id_list if x not in labelled_user_id_list]
# print(len(final_filtered_user_id_list))
# print(final_filtered_user_id_list)


def generate_data_for_label_studio(final_filtered_user_id_list):
    df_chat = pd.read_excel('../all_raw_data/output_with_conversation_sessions_35min.xlsx')
    max_rows = df_chat.shape[0] / 3 # data is labelled by three experts
    # max_rows = 359 # data is labelled by three experts
    random.shuffle(final_filtered_user_id_list)
    df_1 = pd.DataFrame(columns=df_chat.columns)
    df_2 = pd.DataFrame(columns=df_chat.columns)
    df_3 = pd.DataFrame(columns=df_chat.columns)

    for user_id in final_filtered_user_id_list:
        user_rows = df_chat[df_chat['user_id'] == user_id]

        if len(df_1) + len(user_rows) <= max_rows:
            df_1 = pd.concat([df_1, user_rows])
        elif len(df_2) + len(user_rows) <= max_rows:
            df_2 = pd.concat([df_2, user_rows])
        elif len(df_3) + len(user_rows) <= max_rows:
            df_3 = pd.concat([df_3, user_rows])
        else:
            print(f"cannot allocate user_id:{user_id}, df_1 rows: {len(df_1)}, df_2 rows: {len(df_2)}, df_3 rows: {len(df_3)}")
            break
    print(f"df_1 rows: {len(df_1)}, df_2 rows: {len(df_2)}, df_3 rows: {len(df_3)}")

    df_1[['id', 'user_id', 'course_id', 'user_ask_time', 'user_questions', 'chatgpt_answer', 'chatgpt_response_time', 'conversation_session']].to_csv('../all_raw_data/2024S2/additional_need_label_1.csv', index=False)
    df_2[['id', 'user_id', 'course_id', 'user_ask_time', 'user_questions', 'chatgpt_answer', 'chatgpt_response_time', 'conversation_session']].to_csv('../all_raw_data/2024S2/additional_need_label_2.csv', index=False)
    df_3[['id', 'user_id', 'course_id', 'user_ask_time', 'user_questions', 'chatgpt_answer', 'chatgpt_response_time', 'conversation_session']].to_csv('../all_raw_data/2024S2/additional_need_label_3.csv', index=False)

generate_data_for_label_studio(final_filtered_user_id_list)

131
[3572, 3384, 3711, 3612, 3386, 3397, 3761, 3329, 3566, 3542, 3592, 3301, 3365, 3760, 3775, 3420, 3678, 3380, 3764, 3505, 3392, 3662, 3641, 3703, 3611, 3573, 3423, 3444, 3429, 3340, 3271, 3485, 3436, 3331, 3772, 3617, 3776, 3403, 3664, 3632, 3293, 3723, 3753, 3320, 3708, 3649, 3519, 3757, 3390, 3620, 3744, 3373, 3544, 3289, 3663, 3622, 3682, 3585, 3658, 3691, 3659, 3539, 3530, 3496, 3689, 3668, 3587, 3681, 3669, 3674, 3498, 3685, 3326, 3409, 3555, 3565, 3280, 3445, 3372, 3475, 3606, 3424, 3379, 3561, 3394, 3758, 3628, 3613, 3756, 3295, 3482, 3725, 3635, 3680, 3412, 3650, 3471, 3735, 3609, 3493, 3633, 3686, 3610, 3448, 3488, 3728, 3638, 3452, 3608, 3596, 3274, 3273, 3284, 3524, 3518, 3556, 3537, 3586, 3490, 3532, 3713, 3325, 3697, 3466, 3382, 3660, 3666, 3311, 3431, 3522, 3486]
df_1 rows: 358, df_2 rows: 359, df_3 rows: 358


C:\Users\Xinyu\AppData\Local\Temp\ipykernel_12780\2508604302.py:26: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_1 = pd.concat([df_1, user_rows])
C:\Users\Xinyu\AppData\Local\Temp\ipykernel_12780\2508604302.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_2 = pd.concat([df_2, user_rows])
C:\Users\Xinyu\AppData\Local\Temp\ipykernel_12780\2508604302.py:30: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no